In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from collections import Counter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fraudaa/newdf2.csv
/kaggle/input/fraudaa/newtest.csv
/kaggle/input/its-a-fraud/train.csv
/kaggle/input/its-a-fraud/test.csv
/kaggle/input/its-a-fraud/mock_submission.csv


In [3]:
newdf=pd.read_csv('../input/fraudaa/newdf2.csv')

In [4]:
newtest=pd.read_csv('../input/fraudaa/newtest.csv')

In [5]:
dte=pd.read_csv("../input/its-a-fraud/test.csv")


In [6]:
X_train=newdf.drop(['isFraud','TransactionID'],axis=1,inplace=False)
y_train=newdf['isFraud']

In [7]:
X_test=newtest.copy()

In [10]:
X_test=newtest.drop(['TransactionID'],axis=1,inplace=False)

In [25]:
from sklearn.model_selection import RandomizedSearchCV 
from xgboost import XGBClassifier
xgb = XGBClassifier()


param_grid = {
"learning_rate"    : [0.05,0.1, 0.3] ,
 "max_depth"        : [ 12, 15]
}

rs_clf = RandomizedSearchCV(xgb, param_grid, n_iter=6, n_jobs=-1, verbose=20,cv=2, scoring='roc_auc', random_state=42)

In [26]:
rs_clf.fit(X_train,y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2; 1/6] START learning_rate=0.05, max_depth=12............................
[CV 1/2; 1/6] END learning_rate=0.05, max_depth=12;, score=0.939 total time=12.3min
[CV 1/2; 3/6] START learning_rate=0.1, max_depth=12.............................
[CV 1/2; 3/6] END learning_rate=0.1, max_depth=12;, score=0.946 total time=11.9min
[CV 1/2; 5/6] START learning_rate=0.3, max_depth=12.............................
[CV 1/2; 5/6] END learning_rate=0.3, max_depth=12;, score=0.951 total time=11.6min
[CV 2/2; 1/6] START learning_rate=0.05, max_depth=12............................
[CV 2/2; 1/6] END learning_rate=0.05, max_depth=12;, score=0.847 total time=12.4min
[CV 2/2; 3/6] START learning_rate=0.1, max_depth=12.............................
[CV 2/2; 3/6] END learning_rate=0.1, max_depth=12;, score=0.905 total time=12.0min
[CV 2/2; 5/6] START learning_rate=0.3, max_depth=12.............................
[CV 2/2; 5/6] END learning_rate=0.3, 

RandomizedSearchCV(cv=2,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None, max_bin=None,...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None

In [27]:
print(rs_clf.best_params_)

{'max_depth': 15, 'learning_rate': 0.3}


In [32]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state = 42, n_jobs = -1,max_depth=15,scale_pos_weight=75,learning_rate=0.3)
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=15, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [33]:
y_pred = xgb.predict(X_test)


In [34]:
Counter(y_pred)

Counter({0: 144051, 1: 3584})

In [62]:
newtest['IsFraud']=y_pred

In [63]:
neww=newtest[['TransactionID','IsFraud']]

In [65]:
dte = dte.merge(neww, how='outer',copy=False, on ='TransactionID' )

In [54]:
dte.shape

(147635, 434)

In [66]:
yp=dte['IsFraud']


In [67]:
Counter(yp)

Counter({0: 143392, 1: 4243})

In [68]:
yp.to_csv('ypred32.csv')